In [1]:
# import cv2
import numpy as np
import time
import os
import mediapipe as mp
from matplotlib import pyplot as plt
import cv2

In [2]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utils

In [3]:
def mp_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    res = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,res

In [4]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec(color=(142,110,60)),mp_drawing.DrawingSpec(color=(142,110,60)))
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(142,110,60)),mp_drawing.DrawingSpec(color=(142,110,60)))
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,mp_drawing.DrawingSpec(color=(142,110,60)),mp_drawing.DrawingSpec(color=(142,110,60)))    
    return image

# Extracting Keypoints for Data Collection

In [5]:
def extract_keypoints(result):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in result.pose_landmarks.landmark] if result.pose_landmarks else np.zeros(33*4)).flatten()
    lh = np.array([[res.x,res.y,res.z] for res in result.left_hand_landmarks.landmark] if result.left_hand_landmarks else np.zeros(21*3)).flatten()
    rh = np.array([[res.x,res.y,res.z] for res in result.right_hand_landmarks.landmark] if result.right_hand_landmarks else np.zeros(21*3)).flatten()
    return np.concatenate([pose,lh,rh])

In [6]:
path = os.path.join('KeyPoint_Data')
actions = np.array(os.listdir(path))
print(actions)
act_len=len(actions)
no_sequences = 30 #50 vids for each action
sequence_length = 30 #each vid 50 frame length

['eat' 'hello' 'help' 'no' 'please' 'thanks' 'what' 'yes']


In [7]:
for act in actions:
    try:
        os.mkdir(path+'/'+act)
        for i in range(no_sequences):
            os.mkdir(path+'/'+act+'/'+str(i))
    except:
        pass

In [7]:
# collections
cap = cv2.VideoCapture(0)
x=0
act="thanks"
flag=False
with mp_holistic.Holistic(min_detection_confidence=0.6,min_tracking_confidence=0.8) as holistic:
    while cap.isOpened():
        for no in range(no_sequences):
            for fr in range(sequence_length):
                #read frame
                success,frame = cap.read()

                #detection         
                image,res = mp_detection(frame,holistic)

                #draw landmark
                frame = draw_landmarks(image,res)
                
                if fr==0:
                    cv2.putText(frame,'Starting Collection',(120,200),cv2.FONT_HERSHEY_SIMPLEX,1,(170,255,0),2,cv2.LINE_AA)
                    cv2.putText(frame,f"Action:{act} Vid:{no}",(15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(170,255,0),2,cv2.LINE_AA)
                    cv2.imshow("Feed",frame)
                    cv2.waitKey(3000)
                else:
                    cv2.putText(frame,f"Action:{act} Vid:{no}",(15,12),cv2.FONT_HERSHEY_SIMPLEX,1,(170,255,0),2,cv2.LINE_AA)
                    cv2.imshow("Feed",frame)
                kp = extract_keypoints(res)
                npy_path = os.path.join("KeyPoint_Data",act,str(no),str(fr))
                np.save(npy_path,kp)

                #breaking/exit feed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    flag=True
                    break
                
        if no==no_sequences-1 or flag:
            cap.release()
            cv2.destroyAllWindows()
            break